In [83]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
from dotenv import load_dotenv


In [84]:
# Read the data from incidents.csv and print the number of incidents per day
df = pd.read_csv("incidents.csv")

In [85]:
#df.describe()
df.info()

# Drop the rows with missing values
df = df.dropna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          61 non-null     object 
 1   apparent_temperature        38 non-null     float64
 2   code                        61 non-null     int64  
 3   date                        61 non-null     object 
 4   description                 61 non-null     object 
 5   dew_point_2m                38 non-null     float64
 6   endTime                     33 non-null     object 
 7   et0_fao_evapotranspiration  38 non-null     float64
 8   hour                        61 non-null     int64  
 9   iconCategory                61 non-null     int64  
 10  is_day                      61 non-null     float64
 11  latitude                    61 non-null     float64
 12  longitude                   61 non-null     float64
 13  magnitudeOfDelay            61 non-nu

In [86]:
# Get the labels by getting the difference between start and end time
y = pd.to_datetime(df["endTime"]) - pd.to_datetime(df["startTime"])

# Remove the start time and end time columns
df = df.drop(["startTime", "endTime"], axis=1)

# Change the date to a seperate month
df["month"] = pd.to_datetime(df["date"]).dt.month
df = df.drop(["date"], axis=1)

df = df.drop(["type"], axis=1)

In [87]:
# # tokenize description
# vectorizer = CountVectorizer()
# df["description"]= vectorizer.fit_transform(df["description"])

# # Tokenize feature
# df = pd.get_dummies(df, columns=["feature"])

df = df.drop(["description"], axis=1)
df.drop(["id"], axis=1, inplace=True)


In [88]:
X = df.values
y = y.values

# For each value in y, convert it to minutes
for i in range(len(y)):
    y[i] = y[i] / 60000

# Convert y to an integer64 list
y = y.astype("int64")
y = np.log(y)

[28.47626258 28.47626258 28.47626258 28.47626258 27.03769188 26.4679823
 26.9153829  26.9153829  28.06821032 27.54588253 27.54588253 27.434713
 23.96897832 23.96897832 27.03769188 28.05769962]


In [89]:
# Try a linear regression model
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

# train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

-3.6973069419618714

In [90]:
# Try a NN model 
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp.fit(X_train, y_train)
mlp.score(X_test, y_test)

Iteration 1, loss = 40001.78491119
Iteration 2, loss = 112659918617817126505807872.00000000
Iteration 3, loss = 1909474499678885632239807353407597046292846939442813956133589012798469582948954931371515909166394704531812557508664655416099251815834451968.00000000
Iteration 4, loss = inf
Iteration 5, loss = nan
Iteration 6, loss = nan
Iteration 7, loss = nan
Iteration 8, loss = nan
Iteration 9, loss = nan
Iteration 10, loss = nan
Iteration 11, loss = nan
Iteration 12, loss = nan
Iteration 13, loss = nan
Iteration 14, loss = nan
Iteration 15, loss = nan
Iteration 16, loss = nan
Iteration 17, loss = nan
Iteration 18, loss = nan
Iteration 19, loss = nan
Iteration 20, loss = nan
Iteration 21, loss = nan
Iteration 22, loss = nan
Iteration 23, loss = nan
Iteration 24, loss = nan
Iteration 25, loss = nan
Iteration 26, loss = nan
Iteration 27, loss = nan
Iteration 28, loss = nan
Iteration 29, loss = nan
Iteration 30, loss = nan
Iteration 31, loss = nan
Iteration 32, loss = nan
Iteration 33, loss =

C:\Users\ merlijn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\ merlijn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\ merlijn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Iteration 770, loss = nan
Iteration 771, loss = nan
Iteration 772, loss = nan
Iteration 773, loss = nan
Iteration 774, loss = nan
Iteration 775, loss = nan
Iteration 776, loss = nan
Iteration 777, loss = nan
Iteration 778, loss = nan
Iteration 779, loss = nan
Iteration 780, loss = nan
Iteration 781, loss = nan
Iteration 782, loss = nan
Iteration 783, loss = nan
Iteration 784, loss = nan
Iteration 785, loss = nan
Iteration 786, loss = nan
Iteration 787, loss = nan
Iteration 788, loss = nan
Iteration 789, loss = nan
Iteration 790, loss = nan
Iteration 791, loss = nan
Iteration 792, loss = nan
Iteration 793, loss = nan
Iteration 794, loss = nan
Iteration 795, loss = nan
Iteration 796, loss = nan
Iteration 797, loss = nan
Iteration 798, loss = nan
Iteration 799, loss = nan
Iteration 800, loss = nan
Iteration 801, loss = nan
Iteration 802, loss = nan
Iteration 803, loss = nan
Iteration 804, loss = nan
Iteration 805, loss = nan
Iteration 806, loss = nan
Iteration 807, loss = nan
Iteration 80

C:\Users\ merlijn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


ValueError: Solver produced non-finite parameter weights. The input data may contain large values and need to be preprocessed.